# Convirtiendo datos a ttl
:D


In [ ]:
import pandas as pd # no quitar, se rompe el proyectoimport pandas as pd

In [ ]:
#Leemos TODO PORQUE SI
circuits = pd.read_csv('data/circuits.csv')
constructorResults = pd.read_csv('data/constructor_results.csv')
constructors = pd.read_csv('data/constructors.csv')
constructorStandings = pd.read_csv('data/constructor_standings.csv')
drivers = pd.read_csv('data/drivers.csv')
driverStandings = pd.read_csv('data/driver_standings.csv')
lapTimes = pd.read_csv('data/lap_times.csv')
pitStops = pd.read_csv('data/pit_stops.csv')
qualifying = pd.read_csv('data/qualifying.csv')

In [ ]:
#Escribimos en el archivo data lo primero de la salida
with open('data.txt', 'w') as file:
    #Vaciamos el archivo
    file.write("""@prefix f1: <http://ex.org/f1#>.
@prefix owl: <http://www.w3.org/2002/07/owl#>.
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>.
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>. 
@prefix xsd: <http://www.w3.org/2001/XMLSchema#>.\n""")
    
print("""
PARA EL RESTO DEBEN ESCRIBIR CON
with open('data.txt', 'a') as file:
    file.write("LO QUE SEA")
""")

In [ ]:
class Sing:
    _instances = {}  # Dictionary to store instances by their value (string)
    
    def __new__(cls, value):
        # If the instance doesn't exist for the given value, create one
        if value not in cls._instances:
            instance = super(Sing, cls).__new__(cls)
            instance.value = value  # Assign the value to the instance
            cls._instances[value] = instance  # Store the instance by value
        return cls._instances[value]  # Return the stored instance for the value
    
    def __repr__(self):
        return f"Sing({self.value!r})"

# Testing the Singleton behavior based on string values
a1 = Sing("A")
a2 = Sing("A")
b1 = Sing("B")

# Identity comparison (using 'is')
print(a1 is a2)  # Output: True, both are the same instance
print(a1 is b1)  # Output: False, different instances because they have different values

print(f"{type(a1)=}")

In [30]:
# print("nueva, notacion usando las siguientes data classes, esta para \n1 mantener la ontologia limpia es decir no tener duplicas de definicion de algo \n2 hacer un solo flush en el archivo ttl los datos, lo que implicara una mejora en el tiempo de la generacion del archivo")

from dataclasses import dataclass, field
from typing import List, Union, Dict

@dataclass
class RDFPredicate:
    property: str
    objects: List[str]

@dataclass
class RDFStatement:
    _instances = {}
    subject: str
    predicates: dict[str, list[str]] = field(default_factory=dict)

    def __new__(cls, *args, **kwargs):
        # subject expected to either be the first args or with the key "subject"
        subject = kwargs.get("subject", args[0] if args else None)

        if not subject:
            raise ValueError("RDFStatement requires a 'subject' argument.")
        
        if subject not in cls._instances:
            new_instance = super(RDFStatement, cls).__new__(cls)
            new_instance.subject = subject
            new_instance.predicates = {}
            cls._instances[subject] = new_instance
        
        return cls._instances[subject]

    def __init__(self, subject: str, predicate: Union[str, RDFPredicate, List[RDFPredicate]], objects: Union[str, List[str]]):
        if isinstance(predicate, str):
            is_objects_str = isinstance(objects, str)
            if not (is_objects_str or all(isinstance(o, str) for o in objects)):
                raise Exception(f"RDFStatement: {type(objects)} expected Union[str, List[str]]")
            if is_objects_str:
                objects = [objects]
            self._insert_on_property(predicate, objects)
        
        elif isinstance(predicate, RDFPredicate):
            self._insert_on_property(predicate.property, predicate.objects)

        elif isinstance(predicate, list):
            if not all(isinstance(p, RDFPredicate) for p in predicate):
                raise Exception(f"RDFStatement: {type(predicate)=} expected List[RDFPredicate]")
            for rdfp in predicate:
                self._insert_on_property(rdfp.property, rdfp.objects)

        else:
            raise Exception(f"RDFStatement: {type(predicate)=} expected Union[str, RDFPredicate, List[RDFPredicate]]")
        
    def _insert_on_property(self, property: str, objects: list[str]):
        if property in self.predicates:
            for obj in objects:
                if obj not in self.predicates[property]:
                    self.predicates[property].append(obj)
        else:
            self.predicates[property] = objects
    
    def to_ttl(self) -> str:
        ret = self.subject + " "
        for property, objects in self.predicates.items():
            ret += property + " " + ", ".join(objects) + ";\n\t"
        ret = ret[:-3]
        ret += "."
        return ret

A = RDFStatement("ex:A", "a", "ex:xd")
RDFStatement("ex:A", ":?", ["ex:nah", "ex:na2"])
print(A.to_ttl())

ex:A a ex:xd;
	:? ex:nah, ex:na2.


In [42]:
RDFStatement("ex:A", ":T", ["ex:nah", "ex:na2"])
print(A.to_ttl())

ex:A a ex:xd;
	:? ex:nah, ex:na2;
	:T ex:nah, ex:na2.


In [ ]:
property 

class MyClass:
    def __new__(cls, *args, **kwargs):
        print("Calling __new__")
        instance = super(MyClass, cls).__new__(cls)
        print(args, kwargs)
        args[0].append(2)
        return instance

    def __init__(self, x, y):
        print("Calling __init__")
        print(x, y)
        self.x = x
        self.y = y

MyClass([1], y=2)

# Generar RDF Nationality

In [ ]:
# TODO: HACER LA WEA
with open('data.txt', 'a') as file:
    file.write("f1:Country rdf:type owl:Class.\n")

unique_countries = drivers['nationality'].unique()

with open('data.txt', 'a') as file:
    for country in unique_countries:
        #Evitamos que hayan " "en el string y los cambiamos por "_" para que no haya problemas
        country = country.replace(" ", "_")
        file.write(f"f1:{country} rdf:type f1:Nationality .\n")

# Generar RDF Drivers

In [ ]:
# Tenemos las nacionalidades de los pilotos, ahora crearemos el resto de datos

# Creamos la clase f1 Driver

with open('data.txt', 'a') as file:
    file.write("""f1:Driver rdf:type owl:Class;
                rdfs:comment 'A class representing an F1 driver'@en .\n""")
    
# Añadimos cada driver con su nacionalidad correspondiente:
# Por cada driver, añadimos su nombre, su fecha de nacimiento, su nacionalidad y su número de piloto

#Muchos drivers number son simplemente /N, eliminar?
with open('data.txt', 'a') as file:
    for index, row in drivers.iterrows():
        file.write(f"\nf1:{row['driverRef']} rdf:type f1:Driver;\n")
        file.write(f'rdfs:label "{row["forename"]} {row["surname"]}"@en;\n')
        if row['code'] != r'\N':
            file.write(f"f1:code '{row['code']}';\n")
        nationality = row['nationality'].replace(" ", "_")
        file.write(f"f1:Nationality f1:{nationality};\n")
        if row['number'] != r'\N':
            file.write(f"f1:number '{row['number']}'^^xsd:integer;\n")
        
        file.write(f"f1:dateOfBirth '{row['dob']}'^^xsd:date .\n") 


# Generar RDF Pistas

In [ ]:
circuit_ontology = """f1:Circuit a rdf:Class, owl:Class;
    rdfs:label 'A F1 Circuit'@en;
    rdfs:comment 'A class representing a F1 circuit'@en;

f1:City a rdf:Class, owl:Class;
    rdfs:label 'A City'@en;
    rdfs:comment 'A City in the world related to f1'@en;

f1:hostCity a rdf:Property;
    rdfs:label 'City'@en;
    rdfs:comment 'a City hosting a F1 circuit'@en;
    rdfs:domain f1:Circuit;

f1:hostCountry a rdf:Property;
    rdfs:label 'Country'@en;
    rdfs:comment 'a Country hosting a F1 circuit'@en;
    rdfs:domain f1:Circuit;"""

with open('data.txt', 'a') as f:
    f.write(circuit_ontology)
    #Circuits

    circuits_ttl = ""
    for index, row in drivers.iterrows():
        circuits_ttl += f"f1:{row["circuitRef"]} rdf:type f1:Circuit;\n\t"
        circuits_ttl += f"rdfs:label '{row["name"]}'@en;\n\t"
        circuits_ttl += f"f1:City f1:{row["location"]};\n\t"
        circuits_ttl += f"f1: :{row[]};\n\t"
        circuits_ttl += f"f1: :{row[]};\n\t"


# Generar RDF Constructores

In [ ]:
with open('data.txt', 'a') as file:
    file.write("f1:Constructor rdf:type owl:Class;\nrdfs:comment 'A class representing an F1 constructor'@en .\n")
    
    for index, row in constructors.iterrows():
        file.write(f"""\nf1:{row['constructorRef']} rdf:type f1:Constructor;
                    rdfs:label '{row['name']}'@en;
                    f1:nationality 'f{row["nationality"]}';
                    f1:link '{row["url"]}' .\n""")
                    